In [14]:
import pyft
import pandas as pd
import altair as alt

alt.data_transformers.enable("vegafusion")
M6A_COLOR = "#800080"
M5C_COLOR = "#8B4513"
NUC_COLOR = "#A9A9A9"
MSP_COLOR = "#9370db"

In [15]:
dfm = pyft.utils.read_and_center_footprint_table(
    "../../tests/data/ctcf-footprints.bed.gz"
)
dfm.head(2)

,chrom,motif_start,motif_end,strand,fire_qual,fiber_name,has_spanning_msp,footprinted,start,end,centering_position,centering_strand,type
0,chr11,5204946,5204981,+,247,m64076_211222_124721/148505307/ccs,True,False,0,8,5204946,+,not-footprinted
1,chr11,5204946,5204981,+,-1,m64076_211222_124721/51053256/ccs,False,False,0,8,5204946,+,not-footprinted


In [16]:
rgns = pd.read_csv("../../tests/data/ctcf.bed.gz", sep="\t", header=None)
rgns.columns = ["chrom", "start", "end", "name", "score", "strand", "aa"]
fiberbam = pyft.Fiberbam("../../tests/data/ctcf.bam")
centers = []
z = None
for idx, rgn in rgns.iterrows():
    region = (rgn["chrom"], rgn["start"], rgn["end"])
    z = pyft.utils.region_to_centered_df(
        fiberbam, region, strand=rgn["strand"], max_flank=500
    )
    centers.append(z)
centers[0].head(2)

[2024-03-12T04:34:16Z INFO  pyft::fiberdata] 181 records fetched in 0.01s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata made for 181 records in 0.13s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata centered for 181 records in 0.02s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] 172 records fetched in 0.11s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata made for 172 records in 0.09s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata centered for 172 records in 0.02s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] 171 records fetched in 0.12s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata made for 171 records in 0.10s
[2024-03-12T04:34:16Z INFO  pyft::fiberdata] Fiberdata centered for 171 records in 0.02s
[2024-03-12T04:34:17Z INFO  pyft::fiberdata] 125 records fetched in 0.13s
[2024-03-12T04:34:17Z INFO  pyft::fiberdata] Fiberdata made for 125 records in 0.08s
[2024-03-12T04:34:17Z INFO  pyft::fiberdata] Fiberdata centered for 125 records in 0.02s
[2024-03

,chrom,fiber_start,fiber_end,fiber_name,strand,type,start,end,qual,centering_position,centering_strand
0,chr11,5184260,5205600,m64076_211222_124721/148505307/ccs,+,msp,-399,-361,0,5204946,+
0,chr11,5184260,5205600,m64076_211222_124721/148505307/ccs,+,msp,-225,-160,0,5204946,+


In [17]:
both_dfs = pd.concat(centers + [dfm], axis=0).reset_index(drop=True)
both_dfs.tail(2)
both_dfs["type"].value_counts()
# get the average number of m6a sites per fiber_name
both_dfs.groupby(["centering_position", "fiber_name"])[
    "type"
].value_counts().unstack().fillna(0).mean(axis=1)
# drop all NA from the both_dfs
will_use = [
    "chrom",
    "centering_position",
    "centering_strand",
    "start",
    "end",
    "type",
    "fiber_name",
]
both_dfs[will_use].dropna()
both_dfs.query("type == 'footprinted'")[will_use]
dfm[will_use]

,chrom,centering_position,centering_strand,start,end,type,fiber_name
0,chr11,5204946,+,0,8,not-footprinted,m64076_211222_124721/148505307/ccs
1,chr11,5204946,+,0,8,not-footprinted,m64076_211222_124721/51053256/ccs
2,chr11,5204946,+,0,8,not-footprinted,m64076_211222_124721/62391018/ccs
3,chr11,5204946,+,0,8,not-footprinted,m64076_211222_124721/97191992/ccs
4,chr11,5204946,+,0,8,not-footprinted,m64076_211222_124721/99419016/ccs
...,...,...,...,...,...,...,...
10320,chr19,45817350,+,29,35,footprinted,m64076_211222_124721/157222001/ccs
10321,chr19,45817350,+,29,35,footprinted,m64076_211222_124721/65339699/ccs
10322,chr19,45817350,+,29,35,not-footprinted,m64076_211222_124721/6882497/ccs
10323,chr19,45817350,+,29,35,not-footprinted,m64076_211222_124721/31394454/ccs


In [21]:
def make_chart(
    in_df,
    m6a_color=M6A_COLOR,
    m5c_color=M5C_COLOR,
    nuc_color=NUC_COLOR,
    msp_color=MSP_COLOR,
    auto_sort=True,
):
    dfm = (
        in_df.copy()[
            [
                "chrom",
                "centering_position",
                "centering_strand",
                "start",
                "end",
                "type",
                "fiber_name",
            ]
        ]
        .dropna()
        .reset_index(drop=True)
        .infer_objects()
    )  # .query("centered_position_type != '5mC'")

    if auto_sort:
        z = (
            dfm.groupby(
                [
                    "chrom",
                    "fiber_name",
                    "centering_position",
                    "centering_strand",
                    "type",
                ]
            )
            .size()
            .reset_index(name="count")
        )

        # combine the centered_position_type and count into a set of wide columns
        z = (
            z.pivot_table(
                index=["fiber_name", "centering_position", "centering_strand"],
                columns="type",
                values="count",
            )
            .reset_index()
            .fillna(0)
        )

        # join z with both_dfs
        dfm = dfm.merge(
            z, on=["fiber_name", "centering_position", "centering_strand"], how="left"
        )

        # sort on footprinted, m6a, and msp if they exist
        sort_cols = ["footprinted", "m6a", "msp"]
        # filter sort_cols to only include columns that exist in the dataframe
        sort_cols = [x for x in sort_cols if x in dfm.columns]
        # sort
        dfm.sort_values(
            ["chrom", "centering_position", "centering_strand"] + sort_cols,
            inplace=True,
            ascending=[True, True, True, False, False, False],
        )

    dfm["region"] = (
        dfm["chrom"]
        + ":"
        + dfm["centering_position"].astype(str)
        + " "
        + dfm["centering_strand"]
    )

    # set the colors
    color_m6a = alt.param(value=m6a_color, bind=alt.binding(input="color", name="m6a"))
    color_5mc = alt.param(value=m5c_color, bind=alt.binding(input="color", name="5mC"))
    color_nuc = alt.param(value=nuc_color, bind=alt.binding(input="color", name="nuc"))
    color_msp = alt.param(value=msp_color, bind=alt.binding(input="color", name="msp"))
    color_fp = alt.param(
        value="green", bind=alt.binding(input="color", name="footprinted")
    )
    color_not_fp = alt.param(
        value="lightgray", bind=alt.binding(input="color", name="not-footprinted")
    )

    domain = ["5mC", "m6a", "nuc", "msp", "footprinted", "not-footprinted"]
    range_ = [color_5mc, color_m6a, color_nuc, color_msp, color_fp, color_not_fp]
    opacity = dict(zip(domain, [1.0, 1.0, 0.25, 0.25, 0.1, 0.1]))

    # add opacity column to the dataframe
    dfm = dfm.assign(opacity=dfm["type"].map(opacity))

    input_dropdown = alt.binding_select(
        # Add the empty selection which shows all when clicked
        options=dfm.region.unique(),
        name="Region: ",
    )

    selection = alt.selection_point(
        fields=["region"],
        bind=input_dropdown,
        value=dfm.region[0],
    )

    bind_range_w = alt.binding_range(min=200, max=1600, name="Chart width: ")
    param_width = alt.param("width", bind=bind_range_w)
    bind_range_h = alt.binding_range(min=200, max=1600, name="Chart height: ")
    param_height = alt.param("height", bind=bind_range_h)

    type_selection = alt.selection_point(fields=["type"], bind="legend")

    base = (
        alt.Chart(dfm)
        .encode(
            x="start:Q",
            x2="end:Q",
            color=alt.Color("type:O").scale(domain=domain, range=range_),
            y=alt.Y("fiber_name:O", sort=None),
            opacity=alt.Opacity("opacity"),
            # opacity=alt.condition(type_selection, "opacity", alt.value(0))
        )
        .transform_filter(
            selection,
        )
    )  # .add_params(
    #    type_selection
    # )

    if "footprinted" in dfm["type"].unique():
        # draw vertical lines at the start and end of the footprints
        fp_df = dfm[["start", "end", "type", "region"]].query("type == 'footprinted'")
        # pivot the start and end columns into a long format
        fp_df = (
            fp_df.melt(
                id_vars=["type", "region"],
                value_vars=["start", "end"],
                value_name="position",
            )
            .reset_index(drop=True)
            .drop_duplicates()
        )

        footprint_lines = (
            alt.Chart(fp_df)
            .encode(
                x="position:Q",
                # y=alt.Y('fiber_name:O', sort=None),
            )
            .mark_rule(
                color="black",
                strokeWidth=0.75,
                strokeDash=[3, 3],
                opacity=0.75,
            )
            .transform_filter(selection)
        )
        chart = base.mark_rect() + footprint_lines
    else:
        chart = base.mark_rect()

    chart = (
        chart.properties(width=800, height=800)
        .add_params(
            selection,
            param_width,
            param_height,
            color_m6a,
            color_5mc,
            color_nuc,
            color_msp,
            color_fp,
            color_not_fp,
        )
        .interactive()
    )

    return chart

In [22]:
make_chart(both_dfs).save("/Users/mrvollger/Desktop/chart.html")

In [23]:
both_dfs.query("centering_position == 5204946").type.value_counts()

type
m6a                18423
nuc                  984
msp                  893
not-footprinted      829
5mC                  109
footprinted           76
Name: count, dtype: int64